# Batch identify languages in the post


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
from pathlib import Path

from be.data.utils import get_resource
from be.social.insta.loader import InstaLoader
from loguru import logger as lg
from py3langid.langid import MODEL_FILE, LanguageIdentifier

In [ ]:
# normalization of probabilities to an interval between 0 and 1
identifier_norm = LanguageIdentifier.from_pickled_model(MODEL_FILE, norm_probs=True)
# identifier_norm.classify("This should be enough text.")
# ("en", 1.0)
# identifier_norm.rank("This should be enough text.")

In [ ]:
posts_fol = get_resource("ig_fol") / "posts"

In [ ]:
TRANSCRIPT_MODEL_TYPE = "medium"

In [ ]:
il = InstaLoader("")

In [ ]:
posts_fol_iter = list(posts_fol.iterdir())
for ip, post_fol in enumerate(posts_fol_iter):
    # get the shortcode
    shortcode = post_fol.name
    lg.info(f"{ip}/{len(posts_fol_iter)} {post_fol} {shortcode}")

    # check if language is already detected
    lang_fp = post_fol / f"language.json"
    if lang_fp.exists():
        lg.info(f"Language already detected {lang_fp}")
        continue

    # load the post
    ps = il.load_post(shortcode)

    # load the transcript if available
    transcript_fp = post_fol / f"p_transcript_{TRANSCRIPT_MODEL_TYPE}.txt"
    if not transcript_fp.exists():
        lg.info(f"Transcript not found {transcript_fp}")
    else:
        transcript_txt = ""

    # build the post content
    post_content = f"{ps.caption}\n{transcript_txt}"

    # detect the language
    lang_rank = identifier_norm.rank(post_content)
    lang_rank_top_k = lang_rank[:5]

    # dump the lang rank as json
    lang_rank_json = json.dumps(lang_rank_top_k, indent=4)
    # lg.debug(lang_rank_json)

    # check if the language is not 100% just to see whats going on
    if lang_rank[0][1] < 0.99:
        lg.debug(f"Language not 100% {lang_rank_top_k} {shortcode}")
        continue

    # break